In [5]:
import hashlib
from Crypto.Hash import SHAKE128  # Debes instalar pycryptodome si aún no lo tienes.

In [11]:
def shake128(data, output_len):
    shake = hashlib.shake_128()
    shake.update(data)
    return shake.digest(output_len)

def generate_key_pair(r, m, v):
    # Paso 1: Generar una semilla privada aleatoria de 32 bytes
    private_seed = shake128(b'LUOV-Private-Seed', 32)
    
    # Paso 2: Derivar la semilla pública y la matriz T a partir de la semilla privada
    sponge = SHAKE128.new()
    sponge.update(private_seed)
    public_seed = sponge.read(32)  # Los primeros 32 bytes son la semilla pública
    T = sponge.read((v * m + 7) // 8)  # Matriz T de tamaño v x m
    
    # Paso 3: Generar C, L, y Q1 a partir de la semilla pública
    public_sponge = SHAKE128.new()
    public_sponge.update(public_seed)
    C = public_sponge.read(m * 1)  # Constante C
    L = public_sponge.read(m * (m + v))  # Parte lineal L
    Q1 = public_sponge.read(m * (v * (v + 1) // 2 + v * m))  # Primera parte cuadrática Q1

    # Paso 4: Generar Q2 a partir de T y Q1
    Q2 = find_Q2(Q1, T, m, v)  # Implementa la función find_Q2 según la especificación

    # Llave pública consiste en la semilla pública y Q2
    public_key = (public_seed, Q2)

    # Llave privada es simplemente la semilla privada
    private_key = private_seed

    return public_key, private_key

def find_Q2(Q1, T, m, v):
    Q2 = []
    for k in range(m):
        Pk1 = find_Pk1(k, Q1, v)
        Pk2 = find_Pk2(k, Q1, v, m)
        Pk3 = compute_Pk3(Pk1, Pk2, T, v, m)
        Q2.append(pack_Pk3_to_Q2(Pk3, m))
    return Q2

def find_Pk1(k, Q1, v):
    # Implementa el algoritmo para extraer Pk1 de Q1
    Pk1 = [[0] * v for _ in range(v)]
    column = 0
    for i in range(v):
        for j in range(i, v):
            Pk1[i][j] = Q1[k * (v * (v + 1) // 2 + v * m) + column]
            column += 1
    return Pk1

def find_Pk2(k, Q1, v, m):
    # Implementa el algoritmo para extraer Pk2 de Q1
    Pk2 = [[0] * m for _ in range(v)]
    column = 0
    for i in range(v):
        column += v - i
        for j in range(m):
            Pk2[i][j] = Q1[k * (v * (v + 1) // 2 + v * m) + column]
            column += 1
    return Pk2

def compute_Pk3(Pk1, Pk2, T, v, m):
    # Implementa la fórmula para calcular Pk3 a partir de Pk1, Pk2, y T
    Pk3 = [[0] * m for _ in range(m)]
    for i in range(m):
        for j in range(m):
            Pk3[i][j] = sum(Pk1[p][q] * T[p][j] for p in range(v) for q in range(v)) + \
                        sum(Pk2[p][j] for p in range(v))
    return Pk3

def pack_Pk3_to_Q2(Pk3, m):
    # Empaca Pk3 en la forma correcta para incluirlo en Q2
    Q2_row = []
    for i in range(m):
        for j in range(i, m):
            Q2_row.append(Pk3[i][j])
    return Q2_row

In [15]:
generate_key_pair(7, 57, 197)
#print("Public Key:", public_key)
#print("Private Key:", private_key)

NameError: name 'm' is not defined